In [3]:
import numpy as np
import pandas as pd
import os
import math
from time import time
#定义几个函数
# def find_key(keys,obj):  二分查找user id为obj的用户是否在训练集用户列表keys中出现
# def softmax(vec): 9类别的时长转softmax概率，时长太大，指数溢出暂时不用
# def cnt2pro(vec): 9类别的时长转概率，分别除以9类别时长总和
# def close_txt(id_cnt_files): 一次性关闭多个txt文件，id_cnt_files为文件列表
# def getfiles(mode): 返回文件名列表，mode："train", "val", "test"

def find_key(keys,obj):
    len_keys=len(keys)
    l=0
    r=len_keys-1
    while True:
        if r-l<3:
            break
        mid=(l+r)//2
        if keys[mid]==obj:
            return True
        if keys[mid]<obj:
            l=mid+1
        else:
            r=mid+1
    for k in range(l,r+1):
        if keys[k]==obj:
            return True
    return False    
def softmax(vec):
    return_ans=np.zeros([vec.shape[0]])
    for k in range(vec.shape[0]):
        SUM=0.0
        for n in range(vec.shape[0]):
            SUM+=math.exp(vec[n]-vec[k])
        return_ans=1.0/SUM
    return return_ans
def cnt2pro(vec):
    return_ans=np.zeros([vec.shape[0]])
    SUM=0.0
    for n in range(vec.shape[0]):
        SUM+=vec[n]
    for k in range(vec.shape[0]):
        return_ans[k]=vec[k]/SUM
    return return_ans
def close_txt(id_cnt_files):
    for file in id_cnt_files:
        file.close()
def getfiles(mode):
    def get_name_from_txt(file_name):
        name_id=[]
        f=open(file_name)
        lines=f.readlines()
        for line in lines:
            s=line.split(".jpg")[0]
            s=s.split("/")[-1]
            name_id.append(s)
        f.close()
        return name_id
    return_ans=[]
    if mode=="test":
        for i in range(2):
            path="/root/userfolder/luotao/final/test_visit_"+str(i)+"/"+str(i)+"/"
            files=os.listdir(path)
            for file in files:
                return_ans.append(path+file)
        return_ans.sort()
        return return_ans
    elif mode=="train":
        file_name="/root/userfolder/luotao/final_code/data_txt/train.txt"
    elif mode=="val":
        file_name="/root/userfolder/luotao/final_code/data_txt/val.txt"
    name_id=get_name_from_txt(file_name)
    name2file={}
    for i in range(10):
        path="/root/userfolder/luotao/final/train_visit_"+str(i)+"/"+str(i)+"/"
        files=os.listdir(path)
        for file in files:
            s=file.split(".txt")[0]
            name2file[s]=path+file
    return_ans=[]
    for name in name_id:
        return_ans.append(name2file[name])
    return return_ans

In [4]:
# 对训练集进行处理的代码
# 遍历训练集的原始visit txt文件，分9个类别记录每个用户的总时长
# 输出9个txt，每个文件对应一个类别，每个文件中两列记录用户id和时长
# 分两步进行：第一步：遍历txt，第二步：对于每个类别，依据时长对用户排序，删除时长低的80% 或 90% 的用户，再合并同一类别中的相同用户，时长相加

###################################################第一步
id_cnt_files=[]
for i in range(1,10):
    id_cnt_file=open("id_cnt/id_cnt_"+str(i)+".txt","a+")
    id_cnt_files.append(id_cnt_file)
files=getfiles("train")
all_cnt=len(files)
print(all_cnt)
cnt=0
last_time=time()
for file in files:
    label=int(file.split(".txt")[0].split("/")[-1].split("_")[1])
    table = pd.read_table(file, header=None)
    tv=table.values
    user_num=tv.shape[0]
    temp=""
    for i in range(user_num):
        temp+=str(tv[i,0])+","+str(tv[i,1].count("&")+tv[i,1].count("|")) +"\n"
    id_cnt_files[label-1].write(temp)
    if cnt%1000==0:
        now_time=time()
        print("%.4f%%"%(cnt/all_cnt*100),now_time-last_time)
        last_time=time()
    cnt+=1
close_txt(id_cnt_files)
###################################################第二步
id_cnt_files=[]
for i in range(1,10):
    id_cnt_file=open("id_cnt_del_same/id_cnt_"+str(i)+".txt","a+")
    id_cnt_files.append(id_cnt_file)
    
last_time=time()
for i in range(1,10):
    table = pd.read_table("id_cnt/id_cnt_"+str(i)+".txt",sep=",", header=None)    
    temp=table.values[:,:]
    user_num=temp.shape[0]
    sorted_data=temp[np.lexsort(temp.T)]
    if i in [1,2,7,9]:
        rate=0.9
    else:
        rate=0.8
    first_user=int(user_num*rate)
    user2cnt={}
    last_user=""
    for j in range(first_user,user_num):
        if (j-first_user)%10000==0:
            now_time=time()
            print(i,(j-first_user)/(user_num-first_user),now_time-last_time)
            last_time=time()
        now_user=sorted_data[j,0]
        if last_user==now_user:
            user2cnt[now_user]+=sorted_data[j,1]
        else:
            user2cnt[now_user]=sorted_data[j,1]
            last_user=now_user
    users=user2cnt.keys()
    for user in users:
        id_cnt_files[i-1].write(str(user)+","+str(user2cnt[user])+"\n")
close_txt(id_cnt_files)

350000
0.0000% 0.009298563003540039
0.2857% 10.427159547805786
0.5714% 18.299205780029297
0.8571% 19.612576007843018
1.1429% 18.359830379486084
1.4286% 17.644432306289673
1.7143% 17.662076711654663
2.0000% 17.37197494506836
2.2857% 17.574745416641235
2.5714% 17.233805418014526
2.8571% 17.930272340774536
3.1429% 18.100570678710938
3.4286% 17.86739444732666
3.7143% 18.206965684890747
4.0000% 18.04947304725647
4.2857% 18.825207710266113
4.5714% 17.697680711746216
4.8571% 18.287256240844727
5.1429% 17.1077721118927
5.4286% 17.248005151748657
5.7143% 17.710768222808838
6.0000% 18.126909017562866
6.2857% 17.27529740333557
6.5714% 16.783807039260864
6.8571% 17.810384273529053
7.1429% 17.30814504623413
7.4286% 16.404570817947388
7.7143% 17.240471601486206
8.0000% 17.443984508514404
8.2857% 16.868647813796997
8.5714% 17.678115606307983
8.8571% 16.578973054885864
9.1429% 17.224840879440308
9.4286% 16.869121313095093
9.7143% 17.360151052474976
10.0000% 16.80355668067932
10.2857% 17.64149141311645

85.7143% 39.037628173828125
86.0000% 41.51964569091797
86.2857% 40.32932639122009
86.5714% 39.68999624252319
86.8571% 38.177698612213135
87.1429% 37.23377251625061
87.4286% 39.29861354827881
87.7143% 30.0126051902771
88.0000% 25.102023124694824
88.2857% 27.62096929550171
88.5714% 27.818101167678833
88.8571% 27.4190833568573
89.1429% 28.65182900428772
89.4286% 28.369376182556152
89.7143% 26.65985608100891
90.0000% 28.227421045303345
90.2857% 27.54004693031311
90.5714% 29.388200759887695
90.8571% 30.082537412643433
91.1429% 28.939799547195435
91.4286% 29.951877117156982
91.7143% 31.98427391052246
92.0000% 32.77919888496399
92.2857% 37.145158767700195
92.5714% 32.58781623840332
92.8571% 32.06965780258179
93.1429% 31.895142793655396
93.4286% 37.52700471878052
93.7143% 34.57102823257446
94.0000% 30.506553173065186
94.2857% 30.337074041366577
94.5714% 33.27723550796509
94.8571% 34.271795988082886
95.1429% 41.41309404373169
95.4286% 39.35909914970398
95.7143% 39.42083930969238
96.0000% 33.302

1 0.1518242294118801 0.02467799186706543
1 0.15276723704797873 0.024891138076782227
1 0.15371024468407735 0.024295806884765625
1 0.154653252320176 0.024732589721679688
1 0.15559625995627463 0.0246737003326416
1 0.15653926759237324 0.024987220764160156
1 0.15748227522847189 0.024622440338134766
1 0.15842528286457053 0.02479386329650879
1 0.15936829050066917 0.024923086166381836
1 0.16031129813676778 0.024651288986206055
1 0.16125430577286642 0.025278091430664062
1 0.16219731340896507 0.024610280990600586
1 0.1631403210450637 0.0258791446685791
1 0.16408332868116232 0.024750232696533203
1 0.16502633631726096 0.02437281608581543
1 0.1659693439533596 0.024337053298950195
1 0.16691235158945822 0.023728609085083008
1 0.16785535922555686 0.024561643600463867
1 0.1687983668616555 0.02498769760131836
1 0.16974137449775414 0.0217134952545166
1 0.17068438213385276 0.01600956916809082
1 0.1716273897699514 0.018312454223632812
1 0.17257039740605004 0.016449928283691406
1 0.17351340504214868 0.01786

1 0.3385397413594096 0.019736528396606445
1 0.3394827489955083 0.026273488998413086
1 0.3404257566316069 0.02004861831665039
1 0.3413687642677055 0.020009756088256836
1 0.3423117719038042 0.020276784896850586
1 0.3432547795399028 0.020168542861938477
1 0.3441977871760014 0.019442319869995117
1 0.3451407948121001 0.019812822341918945
1 0.3460838024481987 0.022619247436523438
1 0.34702681008429737 0.01896500587463379
1 0.347969817720396 0.020102739334106445
1 0.3489128253564946 0.019040584564208984
1 0.34985583299259326 0.018431425094604492
1 0.3507988406286919 0.0186309814453125
1 0.3517418482647905 0.019883394241333008
1 0.35268485590088916 0.0188140869140625
1 0.3536278635369878 0.019091367721557617
1 0.3545708711730864 0.01879739761352539
1 0.35551387880918506 0.022129058837890625
1 0.35645688644528367 0.018918752670288086
1 0.35739989408138234 0.018459558486938477
1 0.35834290171748095 0.019329309463500977
1 0.35928590935357957 0.019009113311767578
1 0.36022891698967824 0.0191709995

1 0.5233692380347419 0.0279386043548584
1 0.5243122456708406 0.028129100799560547
1 0.5252552533069392 0.022878646850585938
1 0.5261982609430378 0.018724918365478516
1 0.5271412685791365 0.019246816635131836
1 0.5280842762152351 0.01921248435974121
1 0.5290272838513337 0.021124601364135742
1 0.5299702914874324 0.018824100494384766
1 0.530913299123531 0.018937349319458008
1 0.5318563067596296 0.019894123077392578
1 0.5327993143957283 0.019349336624145508
1 0.5337423220318269 0.0202028751373291
1 0.5346853296679255 0.018918991088867188
1 0.5356283373040241 0.020116806030273438
1 0.5365713449401228 0.019454479217529297
1 0.5375143525762214 0.0198211669921875
1 0.53845736021232 0.018740177154541016
1 0.5394003678484187 0.018131017684936523
1 0.5403433754845173 0.018585205078125
1 0.5412863831206159 0.01797652244567871
1 0.5422293907567146 0.018442153930664062
1 0.5431723983928132 0.018903493881225586
1 0.5441154060289118 0.018578767776489258
1 0.5450584136650105 0.018473386764526367
1 0.54

...

7 0.6748043760601593 0.8238949775695801
7 0.6766531551726529 0.015381336212158203
7 0.6785019342851465 0.014566898345947266
7 0.6803507133976401 0.014518022537231445
7 0.6821994925101337 0.015166521072387695
7 0.6840482716226272 0.018669605255126953
7 0.6858970507351208 0.015812397003173828
7 0.6877458298476143 0.015856504440307617
7 0.6895946089601079 0.015299797058105469
7 0.6914433880726015 0.018222570419311523
7 0.6932921671850951 0.015671968460083008
7 0.6951409462975887 0.030050039291381836
7 0.6969897254100823 0.023927688598632812
7 0.6988385045225759 0.023678302764892578
7 0.7006872836350695 0.015751361846923828
7 0.7025360627475631 0.015279531478881836
7 0.7043848418600567 0.019327878952026367
7 0.7062336209725503 0.0166778564453125
7 0.7080824000850439 0.01654362678527832
7 0.7099311791975375 0.016355276107788086
7 0.711779958310031 0.01656627655029297
7 0.7136287374225246 0.017228364944458008
7 0.7154775165350182 0.017189502716064453
7 0.7173262956475118 0.02021932601928711


8 0.13318324958862449 0.022320032119750977
8 0.1377757754365081 0.02224111557006836
8 0.1423683012843917 0.02190995216369629
8 0.14696082713227532 0.019854307174682617
8 0.1515533529801589 0.013662099838256836
8 0.1561458788280425 0.013077974319458008
8 0.16073840467592612 0.013851165771484375
8 0.16533093052380973 0.015196084976196289
8 0.16992345637169332 0.014550447463989258
8 0.17451598221957693 0.014271736145019531
8 0.17910850806746054 0.013440847396850586
8 0.18370103391534415 0.01263737678527832
8 0.18829355976322773 0.013082027435302734
8 0.19288608561111134 0.03452563285827637
8 0.19747861145899495 0.012490987777709961
8 0.20207113730687853 0.012520551681518555
8 0.20666366315476214 0.012591361999511719
8 0.21125618900264576 0.01328897476196289
8 0.21584871485052937 0.013111114501953125
8 0.22044124069841295 0.019486427307128906
8 0.22503376654629656 0.01906442642211914
8 0.22962629239418017 0.018904924392700195
8 0.23421881824206378 0.017452716827392578
8 0.23881134408994736

9 0.03599885057787684 0.012747526168823242
9 0.0381164300236343 0.012493610382080078
9 0.04023400946939176 0.01241445541381836
9 0.04235158891514923 0.013184547424316406
9 0.04446916836090669 0.013389348983764648
9 0.04658674780666415 0.02167534828186035
9 0.04870432725242161 0.012255668640136719
9 0.05082190669817907 0.012880802154541016
9 0.05293948614393653 0.01250004768371582
9 0.055057065589694 0.013174772262573242
9 0.05717464503545146 0.013393878936767578
9 0.05929222448120892 0.013485193252563477
9 0.06140980392696638 0.012740135192871094
9 0.06352738337272384 0.012748241424560547
9 0.0656449628184813 0.012875080108642578
9 0.06776254226423876 0.012591123580932617
9 0.06988012170999622 0.012601137161254883
9 0.07199770115575369 0.012638568878173828
9 0.07411528060151115 0.012455224990844727
9 0.0762328600472686 0.012083768844604492
9 0.07835043949302607 0.012451648712158203
9 0.08046801893878353 0.01267695426940918
9 0.08258559838454099 0.01344752311706543
9 0.08470317783029846

9 0.4531620013920967 0.020594120025634766
9 0.4552795808378542 0.02057790756225586
9 0.45739716028361166 0.020856857299804688
9 0.4595147397293691 0.0166780948638916
9 0.4616323191751266 0.014183521270751953
9 0.46374989862088406 0.013358831405639648
9 0.46586747806664147 0.013392210006713867
9 0.46798505751239894 0.01372671127319336
9 0.4701026369581564 0.013347148895263672
9 0.4722202164039139 0.013956785202026367
9 0.47433779584967134 0.013829469680786133
9 0.4764553752954288 0.012967348098754883
9 0.4785729547411863 0.012712955474853516
9 0.48069053418694374 0.012869119644165039
9 0.4828081136327012 0.01262044906616211
9 0.4849256930784587 0.012788057327270508
9 0.4870432725242161 0.012843608856201172
9 0.48916085196997355 0.012866973876953125
9 0.491278431415731 0.012844562530517578
9 0.4933960108614885 0.012992620468139648
9 0.49551359030724595 0.014398574829101562
9 0.4976311697530034 0.013275384902954102
9 0.4997487491987609 0.013375997543334961
9 0.5018663286445183 0.012979745

9 0.8893833672181337 0.015401363372802734
9 0.8915009466638912 0.015590906143188477
9 0.8936185261096486 0.015358209609985352
9 0.8957361055554062 0.015654563903808594
9 0.8978536850011636 0.01562666893005371
9 0.8999712644469211 0.0164949893951416
9 0.9020888438926785 0.015465259552001953
9 0.904206423338436 0.015625715255737305
9 0.9063240027841934 0.015414714813232422
9 0.9084415822299509 0.015603780746459961
9 0.9105591616757084 0.021165132522583008
9 0.9126767411214658 0.02290201187133789
9 0.9147943205672233 0.0209805965423584
9 0.9169119000129807 0.01776862144470215
9 0.9190294794587383 0.015697240829467773
9 0.9211470589044957 0.01560354232788086
9 0.9232646383502532 0.016243934631347656
9 0.9253822177960106 0.01680588722229004
9 0.9274997972417681 0.015310049057006836
9 0.9296173766875255 0.015002727508544922
9 0.9317349561332829 0.01535940170288086
9 0.9338525355790405 0.015267133712768555
9 0.9359701150247979 0.019800424575805664
9 0.9380876944705554 0.017564773559570312
9 0

In [5]:
# 测试代码
##################################################从id_cnt_del_same文件夹读取训练集的用户时长，存入9个字典中 user2hours
user2hours=[{} for i in range(9)]
for i in range(1,10):
    label=i
    table = pd.read_table("id_cnt_del_same/id_cnt_"+str(i)+".txt",sep=",", header=None)    
    temp=table.values[:,:]
    user_num=temp.shape[0]
    sorted_data=temp[np.lexsort(temp.T)]
    for j in range(user_num):
        if (j)%50000==0:
            print(i,(j)/(user_num))
        now_user=sorted_data[j,0]
        user2hours[i-1][now_user]=int(sorted_data[j,1])
################################################### 根据user2hours记录老用户的时长，
# 因为字典操作需要保证key值存在，所以需要二分搜索key (用户id)是否是在字典的keys中出现过

id2label=np.zeros([100000])
id2pro=np.zeros([100000,9])
id2num=np.zeros([100000,9])
all_keys=[]  #记录所有训练集用户id，用于后续二分查询
for i in range(9):
    temp=list(user2hours[i].keys())
    temp.sort()
    all_keys.append(temp[:])
files=getfiles("test")
all_cnt=len(files)
print(all_cnt)
cnt=0
last_time=time()
for file in files:
    reg_id=int(file.split(".txt")[0].split("/")[-1])
    table = pd.read_table(file, header=None)
    table2cnt=table.values[:,:]
    user_num=table2cnt.shape[0]
    for i in range(user_num):
        now_user_hours=table2cnt[i,1].count("&")+table2cnt[i,1].count("|")
        table2cnt[i,1]=now_user_hours
    temp=table2cnt[:,:]
    sorted_data=temp[np.lexsort(temp.T)]
    rate=0.5   #剔除单个测试样本中时长较短的用户，剔除比例为rate
    first_user=int(user_num*rate)
    reg_per_label=np.zeros([9])
    for i in range(first_user,user_num):
        now_user=str(sorted_data[i,0])
        hours_per_label=np.zeros([9])
        for j in range(9):
            if find_key(all_keys[j],now_user):
                hours_per_label[j]=user2hours[j][now_user]
        reg_per_label+=hours_per_label
    id2num[reg_id]=reg_per_label
    pro=cnt2pro(reg_per_label)
    pre_label=np.argmax(pro)+1
    if np.sum(reg_per_label)==0:  #该测试样本内老用户为0，用类别 -1 标记缺失
        pre_label=-1
    id2label[reg_id]=pre_label
    id2pro[reg_id]=pro[:]
    if cnt%1000==0:
        now_time=time()
        print("%.4f%%"%(cnt/all_cnt*100),now_time-last_time)
        last_time=time()
    cnt+=1

np.save("id2num_test.npy",id2num)
np.save("id2label_test.npy",id2label)
np.save("id2pro_test.npy",id2pro)

1 0.0
1 0.0053854483892016155
1 0.010770896778403231
1 0.01615634516760485
1 0.021541793556806462
1 0.02692724194600808
1 0.0323126903352097
1 0.03769813872441131
1 0.043083587113612924
1 0.04846903550281454
1 0.05385448389201616
1 0.05923993228121777
1 0.0646253806704194
1 0.07001082905962101
1 0.07539627744882262
1 0.08078172583802423
1 0.08616717422722585
1 0.09155262261642746
1 0.09693807100562908
1 0.1023235193948307
1 0.10770896778403231
1 0.11309441617323393
1 0.11847986456243555
1 0.12386531295163716
1 0.1292507613408388
1 0.1346362097300404
1 0.14002165811924203
1 0.14540710650844363
1 0.15079255489764523
1 0.15617800328684686
1 0.16156345167604846
1 0.1669489000652501
1 0.1723343484544517
1 0.17771979684365333
1 0.18310524523285493
1 0.18849069362205656
1 0.19387614201125816
1 0.1992615904004598
1 0.2046470387896614
1 0.21003248717886303
1 0.21541793556806463
1 0.22080338395726626
1 0.22618883234646786
1 0.2315742807356695
1 0.2369597291248711
1 0.24234517751407272
1 0.247730

4 0.5239627829235289
4 0.5567104568562495
4 0.58945813078897
4 0.6222058047216906
4 0.6549534786544112
4 0.6877011525871317
4 0.7204488265198523
4 0.7531965004525728
4 0.7859441743852934
4 0.818691848318014
4 0.8514395222507345
4 0.8841871961834551
4 0.9169348701161757
4 0.9496825440488962
4 0.9824302179816168
5 0.0
5 0.16837964896210783
5 0.33675929792421566
5 0.5051389468863235
5 0.6735185958484313
5 0.8418982448105392
6 0.0
6 0.013984969514164955
6 0.02796993902832991
6 0.04195490854249487
6 0.05593987805665982
6 0.06992484757082477
6 0.08390981708498974
6 0.0978947865991547
6 0.11187975611331964
6 0.1258647256274846
6 0.13984969514164955
6 0.15383466465581452
6 0.16781963416997947
6 0.18180460368414442
6 0.1957895731983094
6 0.20977454271247434
6 0.22375951222663928
6 0.23774448174080426
6 0.2517294512549692
6 0.26571442076913415
6 0.2796993902832991
6 0.2936843597974641
6 0.30766932931162905
6 0.321654298825794
6 0.33563926833995894
6 0.3496242378541239
6 0.36360920736828883
6 0.3

/root/userfolder/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars


1.0000% 79.01915097236633
2.0000% 95.69696688652039
3.0000% 98.68650555610657
4.0000% 99.14291143417358
5.0000% 86.73436117172241
6.0000% 90.27688956260681
7.0000% 96.05997085571289
8.0000% 87.12667918205261
9.0000% 120.67612171173096
10.0000% 104.12147092819214
11.0000% 121.39000129699707
12.0000% 155.4014596939087
13.0000% 129.3446056842804
14.0000% 127.71373534202576
15.0000% 146.5171093940735
16.0000% 107.48908472061157
17.0000% 101.81839847564697
18.0000% 116.44141292572021
19.0000% 128.2414059638977
20.0000% 134.91093635559082
21.0000% 104.94526815414429
22.0000% 103.08661842346191
23.0000% 105.86006879806519
24.0000% 117.21654224395752
25.0000% 125.27270174026489
26.0000% 126.09950685501099
27.0000% 84.51045203208923
28.0000% 111.63965082168579
29.0000% 113.33638787269592
30.0000% 215.1051959991455
31.0000% 208.8454122543335
32.0000% 212.6269953250885
33.0000% 227.83564591407776
34.0000% 171.38530492782593
35.0000% 160.2130708694458
36.0000% 147.33808851242065
37.0000% 236.82022

In [9]:
# 验证集代码
##################################################从id_cnt_del_same文件夹读取训练集的用户时长，存入9个字典中 user2hours
user2hours=[{} for i in range(9)]
for i in range(1,10):
    label=i
    table = pd.read_table("id_cnt_del_same/id_cnt_"+str(i)+".txt",sep=",", header=None)    
    temp=table.values[:,:]
    user_num=temp.shape[0]
    sorted_data=temp[np.lexsort(temp.T)]
    for j in range(user_num):
        if (j)%50000==0:
            print(i,(j)/(user_num))
        now_user=sorted_data[j,0]
        user2hours[i-1][now_user]=int(sorted_data[j,1])
################################################### 根据user2hours记录老用户的时长，
# 因为字典操作需要保证key值存在，所以需要二分搜索key (用户id)是否是在字典的keys中出现过

id2label=np.zeros([50000])
id2pro=np.zeros([50000,9])
id2num=np.zeros([50000,9])
all_keys=[]  #记录所有训练集用户id，用于后续二分查询
for i in range(9):
    temp=list(user2hours[i].keys())
    temp.sort()
    all_keys.append(temp[:])
files=getfiles("val")
files.sort()
all_cnt=len(files)
print(all_cnt)
cnt=0
acc_cnt=0
last_time=time()
for file in files:
    s=file.split(".txt")[0].split("/")[-1].split("_")
    reg_id=int(s[0])
    true_label=int(s[1])
    table = pd.read_table(file, header=None)
    table2cnt=table.values[:,:]
    user_num=table2cnt.shape[0]
    for i in range(user_num):
        now_user_hours=table2cnt[i,1].count("&")+table2cnt[i,1].count("|")
        table2cnt[i,1]=now_user_hours
    temp=table2cnt[:,:]
    sorted_data=temp[np.lexsort(temp.T)]
    rate=0.5   #剔除单个测试样本中时长较短的用户，剔除比例为rate
    first_user=int(user_num*rate)
    reg_per_label=np.zeros([9])
    for i in range(first_user,user_num):
        now_user=str(sorted_data[i,0])
        hours_per_label=np.zeros([9])
        for j in range(9):
            if find_key(all_keys[j],now_user):
                hours_per_label[j]=user2hours[j][now_user]
        reg_per_label+=hours_per_label
    id2num[cnt]=reg_per_label
    pro=cnt2pro(reg_per_label)
    pre_label=np.argmax(pro)+1
    if np.sum(reg_per_label)==0:  #该测试样本内老用户为0，用类别 -1 标记缺失
        pre_label=-1
    id2label[cnt]=pre_label
    id2pro[cnt]=pro[:]
    if true_label==pre_label:
        acc_cnt+=1
    cnt+=1
    if cnt%1000==0:
        now_time=time()
        print("%.4f%%"%(cnt/all_cnt*100),"val acc ",acc_cnt/cnt,now_time-last_time)
        last_time=time()

np.save("id2num_val.npy",id2num)
np.save("id2label_val.npy",id2label)
np.save("id2pro_val.npy",id2pro)

1 0.0
1 0.0053854483892016155
1 0.010770896778403231
1 0.01615634516760485
1 0.021541793556806462
1 0.02692724194600808
1 0.0323126903352097
1 0.03769813872441131
1 0.043083587113612924
1 0.04846903550281454
1 0.05385448389201616
1 0.05923993228121777
1 0.0646253806704194
1 0.07001082905962101
1 0.07539627744882262
1 0.08078172583802423
1 0.08616717422722585
1 0.09155262261642746
1 0.09693807100562908
1 0.1023235193948307
1 0.10770896778403231
1 0.11309441617323393
1 0.11847986456243555
1 0.12386531295163716
1 0.1292507613408388
1 0.1346362097300404
1 0.14002165811924203
1 0.14540710650844363
1 0.15079255489764523
1 0.15617800328684686
1 0.16156345167604846
1 0.1669489000652501
1 0.1723343484544517
1 0.17771979684365333
1 0.18310524523285493
1 0.18849069362205656
1 0.19387614201125816
1 0.1992615904004598
1 0.2046470387896614
1 0.21003248717886303
1 0.21541793556806463
1 0.22080338395726626
1 0.22618883234646786
1 0.2315742807356695
1 0.2369597291248711
1 0.24234517751407272
1 0.247730

4 0.5239627829235289
4 0.5567104568562495
4 0.58945813078897
4 0.6222058047216906
4 0.6549534786544112
4 0.6877011525871317
4 0.7204488265198523
4 0.7531965004525728
4 0.7859441743852934
4 0.818691848318014
4 0.8514395222507345
4 0.8841871961834551
4 0.9169348701161757
4 0.9496825440488962
4 0.9824302179816168
5 0.0
5 0.16837964896210783
5 0.33675929792421566
5 0.5051389468863235
5 0.6735185958484313
5 0.8418982448105392
6 0.0
6 0.013984969514164955
6 0.02796993902832991
6 0.04195490854249487
6 0.05593987805665982
6 0.06992484757082477
6 0.08390981708498974
6 0.0978947865991547
6 0.11187975611331964
6 0.1258647256274846
6 0.13984969514164955
6 0.15383466465581452
6 0.16781963416997947
6 0.18180460368414442
6 0.1957895731983094
6 0.20977454271247434
6 0.22375951222663928
6 0.23774448174080426
6 0.2517294512549692
6 0.26571442076913415
6 0.2796993902832991
6 0.2936843597974641
6 0.30766932931162905
6 0.321654298825794
6 0.33563926833995894
6 0.3496242378541239
6 0.36360920736828883
6 0.3

/root/userfolder/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars


2.0000% val acc  0.695 92.66069459915161
4.0000% val acc  0.683 128.27018976211548
6.0000% val acc  0.6773333333333333 131.3728892803192
8.0000% val acc  0.67925 120.07485771179199
10.0000% val acc  0.6838 114.9378023147583
12.0000% val acc  0.6853333333333333 117.85466766357422
14.0000% val acc  0.6858571428571428 102.3244686126709
16.0000% val acc  0.682875 121.27261447906494
18.0000% val acc  0.6815555555555556 123.62095332145691
20.0000% val acc  0.6794 119.92616891860962
22.0000% val acc  0.681 116.7556962966919
24.0000% val acc  0.6806666666666666 102.70332884788513
26.0000% val acc  0.6807692307692308 95.01872301101685
28.0000% val acc  0.6815 97.36257982254028
30.0000% val acc  0.6826 93.99433183670044
32.0000% val acc  0.6840625 92.2463927268982
34.0000% val acc  0.6840588235294117 101.4839997291565
36.0000% val acc  0.6823888888888889 107.441654920578
38.0000% val acc  0.6826315789473684 97.47093987464905
40.0000% val acc  0.6825 96.60793566703796
42.0000% val acc  0.68357142